In [ ]:
import pandas as pd
import xlsxwriter

In [ ]:
## read metrics and concat

df_dte = pd.read_excel('dte_metrics.xlsx')
df_nn = pd.read_excel('nn_metrics.xlsx')
df_log = pd.read_excel('log_metrics.xlsx')
df_svm = pd.read_excel('svm_metrics.xlsx')

df = pd.concat([df_dte,df_nn])
df = pd.concat([df,df_log])
df = pd.concat([df,df_svm])

df.to_excel('all_metrics.xlsx')
df.to_pickle('all_metrics.pkl')

In [ ]:
## grouping 

grouped = df.groupby(df.group)
df_abiEltern = grouped.get_group("abiEltern")
df_gender = grouped.get_group("gender")
df_erstsprache = grouped.get_group("erstsprache")
df_buecher = grouped.get_group("buecher")

In [ ]:
## calculate PP, EO, SA, PE for each demographic group

# gender
df_gender = df_gender.drop(columns=['group', 'Unnamed: 0', 'Accuracy'])
df_gender = pd.pivot_table(df_gender, values=["Precision","Recall","AUC","FPR"], index=["model"], columns=["subgroup"])
df_gender['PP'] = df_gender.Precision.girls-df_gender.Precision.boys
df_gender['EO'] = df_gender.Recall.boys-df_gender.Recall.girls
df_gender['SA'] = df_gender.AUC.girls-df_gender.AUC.boys
df_gender['PE'] = df_gender.FPR.boys-df_gender.FPR.girls

# first language
df_erstsprache = df_erstsprache.drop(columns=['group', 'Unnamed: 0', 'Accuracy'])
df_erstsprache = pd.pivot_table(df_erstsprache, values=["Precision","Recall","AUC","FPR"], index=["model"], columns=["subgroup"])
df_erstsprache['PP'] = df_erstsprache.Precision.deutsch-df_erstsprache.Precision.migration
df_erstsprache['EO'] = df_erstsprache.Recall.migration-df_erstsprache.Recall.deutsch
df_erstsprache['SA'] = df_erstsprache.AUC.deutsch-df_erstsprache.AUC.migration
df_erstsprache['PE'] = df_erstsprache.FPR.migration-df_erstsprache.FPR.deutsch

# HLE
df_buecher = df_buecher.drop(columns=['group', 'Unnamed: 0', 'Accuracy'])
df_buecher = pd.pivot_table(df_buecher, values=["Precision","Recall","AUC","FPR"], index=["model"], columns=["subgroup"])
df_buecher['PP'] = df_buecher.Precision.buch1-df_buecher.Precision.buch0
df_buecher['EO'] = df_buecher.Recall.buch0-df_buecher.Recall.buch1
df_buecher['SA'] = df_buecher.AUC.buch1-df_buecher.AUC.buch0
df_buecher['PE'] = df_buecher.FPR.buch0-df_buecher.FPR.buch1

# parental education
df_abiEltern = df_abiEltern.drop(columns=['group', 'Unnamed: 0', 'Accuracy'])
df_abiEltern = pd.pivot_table(df_abiEltern, values=["Precision","Recall","AUC","FPR"], index=["model"], columns=["subgroup"])
df_abiEltern['PP'] = df_abiEltern.Precision.abi-df_abiEltern.Precision.keinAbi
df_abiEltern['EO'] = df_abiEltern.Recall.keinAbi-df_abiEltern.Recall.abi
df_abiEltern['SA'] = df_abiEltern.AUC.abi-df_abiEltern.AUC.keinAbi
df_abiEltern['PE'] = df_abiEltern.FPR.keinAbi-df_abiEltern.FPR.abi

# cleaning
df_buecher = df_buecher.drop(columns=['AUC','Precision','Recall','FPR'])
df_erstsprache = df_erstsprache.drop(columns=['AUC','Precision','Recall','FPR'])
df_gender = df_gender.drop(columns=['AUC','Precision','Recall','FPR'])
df_abiEltern = df_abiEltern.drop(columns=['AUC','Precision','Recall','FPR'])

df_buecher.columns = df_buecher.columns.droplevel(1)
df_erstsprache.columns = df_erstsprache.columns.droplevel(1)
df_gender.columns = df_gender.columns.droplevel(1)
df_abiEltern.columns = df_abiEltern.columns.droplevel(1)

df_buecher = pd.pivot_table(df_buecher, values=["PP","EO","SA","PE"],  columns=["model"])
df_gender = pd.pivot_table(df_gender, values=["PP","EO","SA","PE"], columns=["model"])
df_erstsprache = pd.pivot_table(df_erstsprache, values=["PP","EO","SA","PE"], columns=["model"])
df_abiEltern = pd.pivot_table(df_abiEltern, values=["PP","EO","SA","PE"], columns=["model"])

In [ ]:
"""
function to format results
set two threshols: one at |0.02| in orange and one at |0.05| in red
format all negative values in bold
"""

def threshold001(v, props=''):
    return props if (v > 0.02) or (v < -0.02) else None

def threshold005(v, props=''):
    return props if (v > 0.05) or (v < -0.05) else None

def negativeValue(v, props=''):
    return props if (v < 0) else None

def showTable(df):
    styled = df.style.set_properties(color="black", align="right")\
        .set_properties(**{'background-color': 'white'})\
        .applymap(threshold001, props='color:orange;')\
        .applymap(threshold005, props='color:red;')\
        .applymap(negativeValue, props='font-weight:bold;')
    return styled

In [ ]:
s = showTable(df_gender)
s

In [ ]:
s = showTable(df_erstsprache)
s

In [ ]:
s = showTable(df_abiEltern)
s

In [ ]:
s = showTable(df_buecher)
s

In [ ]:
## save as excel

writer = pd.ExcelWriter('fairness_double.xlsx', engine='xlsxwriter')

df_gender.to_excel(writer, sheet_name='Gender')
df_abiEltern.to_excel(writer, sheet_name='AbiEltern')
df_erstsprache.to_excel(writer, sheet_name='ErstSprache')
df_buecher.to_excel(writer, sheet_name='Buecher')
writer.save()